In [1]:
import os
import torch
import pytorch_lightning as pl
import pandas as pd
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from PIL import Image
from datasets import load_dataset
import torch.nn.functional as F
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

E:\Python\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        item = self.data[idx]
        image = item['image']
        label = float(item['label'])#标签是连续值

        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label, dtype=torch.float32)

In [3]:

class ImageRegression(pl.LightningModule):
    def __init__(self):
        super(ImageRegression,self).__init__()
        self.model = torch.hub.load('pytorch/vision:v0.10.0','resnet50',pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs,1)

    def forward(self, x):
        x = x.float()  # 转换数据类型为float32
        return self.model(x).squeeze(-1)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.mse_loss(outputs, labels)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        rounded_outputs = torch.round(outputs)
        loss = F.mse_loss(rounded_outputs, labels)
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        correct = (rounded_outputs == labels).sum().item()
        total = len(labels)
        accuracy = correct / total
        self.log('val_acc', accuracy, on_epoch=True, prog_bar=True)
        return {"val_loss": loss, "val_acc": accuracy}

#    def on_validation_epoch_end(self):
#        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
#        avg_acc = torch.stack([x['val_acc']for x in outputs]).mean()
        
    def test_step(self,batch,batch_idx):
        images, labels = batch
        outputs = self(images)
        rounded_outputs = torch.round(outputs)
        loss = F.mse_loss(rounded_outputs, labels)
        self.log('test_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        correct = (rounded_outputs == labels).sum().item()
        total = len(labels)
        accuracy = correct / total
        self.log('test_acc', accuracy, on_epoch=True, prog_bar=True)
        return {"test_loss": loss, "test_acc": accuracy}

 #   def on_test_epoch_end(self):
 #       test_results = self.trainer.callback_metrics
 #       test_loss = test_results['test_loss_epoch'] 
#        self.log('test_loss', test_loss, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer


In [4]:
#if __name__ == "__main__":
raw_datasets = load_dataset("Niche-Squad/mock-dots","regression-one-class", download_mode="force_redownload") # 使用你的数据集名称加载数据 

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
checkpoint_callback = ModelCheckpoint(monitor='val_acc',
dirpath = "E:/Files/Checkpoint",  # 设置存储路径
mode='max',          # 如果你的指标是准确率或类似的，可能需要更改为'max'
filename='best-model-{epoch:02d}-{val_acc:.2f}',
save_top_k=1,
verbose=True,# 打印检查点保存信息
save_last=False,      # 如果你还想保存最后一个checkpoint
)


train_dataset = CustomDataset(raw_datasets['train'], transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = CustomDataset(raw_datasets['validation'], transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

test_dataset = CustomDataset(raw_datasets['test'], transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    
model = ImageRegression()
logger = TensorBoardLogger("tb_logs", name="Resnet50_Regression_Classification_batch_size_32_epoch_100_lr_0.001_callbacks_T")#save_dir="your/log/directory"
trainer = pl.Trainer(callbacks=[checkpoint_callback],max_epochs=100,logger = logger)#callbacks=[checkpoint_callback]
trainer.fit(model, train_loader,val_loader)




Generating train split:   0%|          | 0/600 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

Using cache found in C:\Users\34691/.cache\torch\hub\pytorch_vision_v0.10.0
E:\Python\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
E:\Python\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high'

Sanity Checking: 0it [00:00, ?it/s]

...

Epoch 97, global step 1862: 'val_acc' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 98, global step 1881: 'val_acc' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 99, global step 1900: 'val_acc' was not in top 1
`Trainer.fit` stopped: `max_epochs=100` reached.


In [5]:
best_model_path = trainer.checkpoint_callback.best_model_path
best_model = ImageRegression.load_from_checkpoint(best_model_path)
print("Best model path:", best_model_path)
print("Best model:", best_model)

Using cache found in C:\Users\34691/.cache\torch\hub\pytorch_vision_v0.10.0
E:\Python\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
E:\Python\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Best model path: E:\Files\Checkpoint\best-model-epoch=43-val_acc=0.70.ckpt
Best model: ImageRegression(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [6]:
trainer.test(dataloaders=test_loader,ckpt_path=best_model_path) 

Restoring states from the checkpoint path at E:\Files\Checkpoint\best-model-epoch=43-val_acc=0.70.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at E:\Files\Checkpoint\best-model-epoch=43-val_acc=0.70.ckpt
E:\Python\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.testing metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.6100000143051147
     test_loss_epoch        0.7099999785423279
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss_epoch': 0.7099999785423279, 'test_acc': 0.6100000143051147}]